# Detecting Topic Shifts in Conversations with Semantic Analysis

This notebook aims to detect topic shifts in conversations through semantic analysis. By utilizing vector embeddings, we can perceive subtle changes in semantic similarity, which helps identify potential shifts in conversation topics.

In [ ]:
%pip install cohere scikit-learn matplotlib seaborn 


In [ ]:
%pip install semantic-router==0.0.15 

In [ ]:
%set_env COHERE_API_KEY = L3SLQxnOJHhLeAEPx9UrO0YVZpGEsNlDb7ITufYR

### Embedding Model

We will use cohere's recent `embed-english-v3.0` model for generating the embeddings

In [ ]:
import os
import cohere
from semantic_router.utils.splitters import semantic_splitter
from semantic_router.encoders import CohereEncoder
encoder = CohereEncoder(cohere_api_key= os.environ.get("COHERE_API_KEY"))

co = cohere.Client( os.environ.get("COHERE_API_KEY"))


def embed_documents(documents: list[str]) -> list[list[float]]:
    emb = co.embed(
        documents,
        input_type="search_document",
        model="embed-english-v3.0",
    ).embeddings
    return emb

### Confusion Matrix

In [ ]:
convo = [
    "User: Hi there! Can you tell me the current temperature?",
    "Bot: Hello! It's currently 68°F.",
    "User: That's quite pleasant. I'm thinking of going for a run.",
    "Bot: That sounds like a great idea. Do you need any running tips?",
    "User: Yes, how can I improve my stamina?",
    "Bot: There are several ways to improve stamina, including interval training and cross-training.",
    "User: Thanks for the advice! By the way, do you have any book recommendations?",
    "Bot: Sure, what genre are you interested in?",
    "User: I enjoy mystery novels.",
    "Bot: I would recommend 'The Girl with the Dragon Tattoo' by Stieg Larsson.",
    "User: Sounds interesting, I'll check it out. I also need some advice on home gardening.",
    "Bot: Of course, what do you need help with?",
    "User: I'm having trouble growing tomatoes.",
    "Bot: Tomato plants need plenty of sunlight and well-drained soil. Also, make sure you're watering them correctly.",
    "User: I'll keep that in mind. Thanks for your help!",
    "Bot: You're welcome! If you have any other questions, feel free to ask.",
    "User: Will do. Goodbye for now!",
    "Bot: Goodbye! Have a great day!"
]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt


embeddings = embed_documents(convo)
similarity_matrix = cosine_similarity(embeddings)

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    similarity_matrix,
    annot=True,
    cmap="YlGnBu",
    xticklabels=range(1, len(convo) + 1),
    yticklabels=range(1, len(convo) + 1),
)
ax.xaxis.tick_top()  # x axis on top
ax.xaxis.set_label_position("top")
# plt.title('Confusion Matrix between Messages')
plt.xlabel('Message Number')
plt.ylabel('Message Number')
plt.show()

In [ ]:
%pip install tabulate colorama pandas 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from colorama import Fore, Style
from tabulate import tabulate
import pandas as pd


def visualize_topic_segments(segments: dict[str, list]):
    colors = [
        Fore.BLUE,
        Fore.GREEN,
        Fore.RED,
        Fore.YELLOW,
        Fore.MAGENTA,
        Fore.CYAN,
        Fore.WHITE,
    ]
    rows = [
        {
            "Topic": colors[i % 7] + segment + Style.RESET_ALL,
            "Message": colors[i % 7] + message + Style.RESET_ALL,
        }
        for i, (segment, messages) in enumerate(segments.items())
        for message in messages
    ]

    df = pd.DataFrame(rows)
    print(tabulate(df, headers="keys", tablefmt="psql"))

In [ ]:
%pip install semantic-router[local]==0.0.15
%pip install semantic-router[fastembed]

In [ ]:
segments = semantic_splitter(encoder=encoder,docs=convo,threshold=0.7,split_method='consecutive_similarity_drop')
from semantic_router.encoders import FastEmbedEncoder
encoder2 = FastEmbedEncoder()

visualize_topic_segments(segments)

Let's try few more conversations

In [ ]:
convo1 = [
    "User: Hi there! Can you tell me the current temperature?",
    "Bot: Hello! It's currently 68°F.",
    "User: That's quite pleasant. I'm thinking of going for a run.",
    "Bot: That sounds like a great idea. Do you need any running tips?",
    "User: Yes, how can I improve my stamina?",
    "Bot: There are several ways to improve stamina, including interval training and cross-training.",
    "User: Thanks for the advice! By the way, do you have any book recommendations?",
    "Bot: Sure, what genre are you interested in?",
    "User: I enjoy mystery novels.",
    "Bot: I would recommend 'The Girl with the Dragon Tattoo' by Stieg Larsson.",
    "User: Sounds interesting, I'll check it out. I also need some advice on home gardening.",
    "Bot: Of course, what do you need help with?",
    "User: I'm having trouble growing tomatoes.",
    "Bot: Tomato plants need plenty of sunlight and well-drained soil. Also, make sure you're watering them correctly.",
    "User: I'll keep that in mind. Thanks for your help!",
    "Bot: You're welcome! If you have any other questions, feel free to ask.",
    "User: Will do. Goodbye for now!",
    "Bot: Goodbye! Have a great day!"
]

convo2 = [
    "User: Hello! Can you tell me the latest news headlines?",
    "Bot: Hi! Sure, here are the top news headlines for today...",
    "User: That's quite interesting. I'm also looking for some new music to listen to.",
    "Bot: What genre do you prefer?",
    "User: I like pop music.",
    "Bot: You might enjoy the latest album by Dua Lipa.",
    "User: I'll give it a listen. Also, I'm planning a trip and need some travel tips.",
    "Bot: Sure, where are you planning to go?",
    "User: I'm thinking of visiting Italy.",
    "Bot: Italy is a beautiful country. Make sure to visit the Colosseum in Rome and the canals in Venice.",
    "User: Those sound like great suggestions. I also need some help with my diet.",
    "Bot: What kind of diet are you following?",
    "User: I'm trying to eat more protein.",
    "Bot: Include lean meats, eggs, and legumes in your diet for a protein boost.",
    "User: Thanks for the tips! I'll talk to you later.",
    "Bot: You're welcome! Don't hesitate to reach out if you need more help.",
    "User: I appreciate it. Goodbye!",
    "Bot: Goodbye! Take care!"
]

Function to get all conversations by conversation id ordered by timestamp from dataframe

In [ ]:
df = pd.read_csv("output_file.csv")
def get_messages_by_conversation_id(df, conversation_id):
    # Filter the DataFrame based on the given conversation_id
    conversation_df = df[df['conversation_id'] == conversation_id]
    
    # Sort the filtered DataFrame by timestamp if applicable
    if 'timestamp' in conversation_df.columns:
        conversation_df = conversation_df.sort_values(by='timestamp')

    # Extract the 'message' column entries
    messages = conversation_df['message'].tolist()

    return messages


Get 20 Conversations from Client Data for testing

In [ ]:
clientconvo1 = get_messages_by_conversation_id(df, "00c25b77-46f8-41ba-85e8-41704a2bf158")
clientconvo2 = get_messages_by_conversation_id(df, "0ab9dbc5-63b2-4d2a-9f25-23ff595400e2")
clientconvo3 = get_messages_by_conversation_id(df, "1632785b-791a-4771-a8fd-0a660a418b18")
clientconvo4 = get_messages_by_conversation_id(df, "1d0621e2-766d-4a0c-85ca-496470dd922c") #huge conversations
clientconvo5 = get_messages_by_conversation_id(df, "249fc8ca-03fa-423d-828f-0e2b617245f6") #huge conversations
clientconvo6 = get_messages_by_conversation_id(df, "36b2cf32-e05e-4df5-b927-e1d7e99f6025")
clientconvo7 = get_messages_by_conversation_id(df, "39505712-29e8-412b-9511-8f0d09c5025c") #huge conversations
clientconvo8 = get_messages_by_conversation_id(df, "3a409efd-737f-4d07-a6ec-cd8c106eaf26") #huge conversations
clientconvo9 = get_messages_by_conversation_id(df, "457f3529-b0d6-4ebb-a0e1-519007f4de43")
clientconvo10 = get_messages_by_conversation_id(df, "4f47b5a5-65a0-4a0a-ba91-e0490609c487")
clientconvo11 = get_messages_by_conversation_id(df, "5c828b96-27ab-4e20-9fef-4db55b38af33")
clientconvo12 = get_messages_by_conversation_id(df, "687b20de-6880-4c99-9222-d03bfc3ffb7a")
clientconvo13 = get_messages_by_conversation_id(df, "6b3572d7-ac24-4aa1-867c-f2f24ce7823a")
clientconvo14 = get_messages_by_conversation_id(df, "7474c06d-0dee-4b18-9514-5501622490d1")
clientconvo15 = get_messages_by_conversation_id(df, "84e9cc7e-6294-4bb6-a0dd-7479c2b09514") #hugest conversations
clientconvo16 = get_messages_by_conversation_id(df, "875b1e69-7531-4804-97bb-3d9000861998")
clientconvo17 = get_messages_by_conversation_id(df, "938511a5-2f03-482d-b63e-74553681a019")
clientconvo18 = get_messages_by_conversation_id(df, "980f3aaf-87e2-4f9c-a7e0-c511a3f04c9b")
clientconvo19 = get_messages_by_conversation_id(df, "a52070ff-f961-48f3-910a-258fb2a6b8d6")
clientconvo20 = get_messages_by_conversation_id(df, "f1463e09-ab61-4e74-a001-89774e69da16")

Below encoder is Cohere Encoder and encoder2 is Fastembed Encoder

In [ ]:
segments = semantic_splitter(encoder=encoder,docs=clientconvo3,threshold=0.43,split_method='consecutive_similarity_drop')
# segments2 = semantic_splitter(encoder=encoder2,docs=clientconvo15,threshold=0.9,split_method='cumulative_similarity_drop')
# visualize_topic_segments(segments) 
# visualize_topic_segments(segments2)

10 Chat GPT Generated Conversations 

In [ ]:
gptconvo = [     
    "User: Hi there! How's the weather today?",
    "Chatbot: Hello! The weather is sunny with a temperature of 25°C. What's on your mind?",
    "User: Nice! Perfect weather for outdoor activities. Do you follow any sports?",
    "Chatbot: Absolutely! I'm a fan of basketball. What's your favorite sport?",
    "User: Basketball is cool! Speaking of cool, have you seen any good movies lately?",
    "Chatbot: I'm not into movies, but I've heard 'Inception' is quite popular. What genre do you enjoy?",
    "User: 'Inception' is a mind-bender! By the way, do you have any favorite snacks for movie nights?",
    "Chatbot: I don't eat, but popcorn seems to be a classic choice. What's your go-to movie snack?"
]
gptconvo2 = [
    "User: Good morning! How's the weather today?",
    "Chatbot: Good morning! The weather is a bit cloudy with a chance of rain. What can I do for you?",
    "User: Cloudy, huh? That's a change. Do you enjoy outdoor activities in different weather?",
    "Chatbot: I don't have preferences, but outdoor activities can be fun! Any favorite activities you enjoy in specific weather?",
    "User: I love hiking when it's cloudy. By the way, do you follow any sports?",
    "Chatbot: Hiking is a great choice! I'm into soccer. What's your favorite sport, if any?",
    "User: Soccer is exciting! I prefer basketball. Have you watched any good movies lately?",
    "Chatbot: Basketball is intense! I don't watch movies, but 'The Shawshank Redemption' is often recommended. What's your favorite movie genre?",
    "User: 'The Shawshank Redemption' is a classic! I enjoy sci-fi. Speaking of classics, any favorite foods or cuisines?",
    "Chatbot: Classics are timeless! I don't eat, but people seem to love pizza. What's your go-to comfort food?",
    "User: Pizza is a crowd-pleaser! I love pasta as my comfort food. Do you have any favorite pasta dishes?",
    "Chatbot: Pasta is versatile! I've heard people enjoy spaghetti bolognese. What's your favorite pasta recipe?",
    "User: Spaghetti bolognese is a classic choice! I like adding a twist with homemade pesto. Do you have any favorite homemade recipes?",
    "Chatbot: I don't cook, but homemade pesto sounds delicious! Share your recipe if you'd like. Anything else on your mind?",
    "User: Sure, here's the recipe... And not much else. Thanks for the chat!",
    "Chatbot: You're welcome! Feel free to return anytime. Have a great day!"
]

gptconvo3 = [ # Music, Travel, Tech
    "User: Hey! Any favorite music genre you like to listen to?",
    "Chatbot: I don't listen to music, but many enjoy pop. Do you have a preferred genre?",
    "User: Pop is catchy! By the way, do you enjoy traveling? Any favorite destinations?",
    "Chatbot: I don't travel, but people like tropical places. Where's your dream vacation spot?",
    "User: I love tropical destinations! Hawaii is on my bucket list. Speaking of tech, any latest gadgets you find interesting?",
    "Chatbot: Gadgets are fascinating! I've heard about the latest smartphones. What tech gadgets catch your eye?",
    "User: Smartphones are evolving fast! I'm eyeing the new model. Do you follow any tech blogs or news?",
    "Chatbot: I don't follow news, but tech blogs are popular. Which ones do you find most informative?",
    "User: I often read TechCrunch and Wired. They provide great insights into upcoming tech trends. Have you heard of any exciting tech advancements recently?",
    "Chatbot: I'm not updated on tech advancements, but those sound like reliable sources. What's the most intriguing tech development you've come across lately?",
    "User: Quantum computing has been making waves. The possibilities are mind-boggling! Ever looked into quantum tech?",
    "Chatbot: Quantum computing is fascinating! It's a complex field, but the potential is enormous. Do you have a favorite tech concept or innovation?",
    "User: AI-driven innovations always catch my attention. The way AI is shaping various industries is incredible. Any AI applications that you find intriguing?",
    "Chatbot: AI is indeed transformative! Natural language processing in AI is quite interesting. It enables more human-like interactions. What's your take on the future of AI in daily life?",
    "User: I believe AI will become increasingly integrated into our daily routines, making tasks more efficient. It's exciting to see the possibilities unfold. Anything else in the tech world that excites you?",
    "Chatbot: The potential of augmented reality (AR) is intriguing. Imagine the immersive experiences it could bring! What are your thoughts on AR applications?"
]

gptconvo4 = [ # Books, Fitness, Food
    "User: Hi! Do you enjoy reading? Any favorite book genre?",
    "Chatbot: I don't read, but many like mystery novels. What's your favorite book genre?",
    "User: Mysteries are intriguing! On a different note, do you have a fitness routine? Any favorite exercises?",
    "Chatbot: I don't exercise, but people like jogging. What's your favorite workout routine?",
    "User: Jogging is great! I prefer weightlifting. Shifting gears, any favorite cuisines or dishes?",
    "Chatbot: I don't eat, but people love sushi. What's your go-to cuisine when dining out?",
    "User: Sushi is a delicacy! I enjoy Italian cuisine. Any favorite Italian dishes you've heard of?",
    "Chatbot: Italian food is popular! I've heard about risotto. What's your favorite Italian dish?",
    "User: Risotto is delicious! I'm a fan of seafood risotto. Speaking of food, do you have any dietary preferences or restrictions?",
    "Chatbot: I don't have dietary preferences, but people often follow vegetarian or vegan diets. Are you conscious of your dietary choices?",
    "User: I try to maintain a balanced diet. It's important to get a variety of nutrients. By the way, any favorite cooking styles or techniques you've come across?",
    "Chatbot: Cooking styles vary, but sous-vide cooking is gaining popularity. It involves precise temperature control. What cooking techniques do you find interesting?",
    "User: Sous-vide is a game-changer! I also find grilling and smoking fascinating. The flavors they impart to food are unique. Ever tried your hand at cooking?",
    "Chatbot: I don't cook, but I've heard about the joy people find in creating dishes. What's your favorite dish to cook at home?",
    "User: I love making homemade pizza. The process is therapeutic, and you can customize the toppings. Do you have a favorite type of cuisine you'd like to try if you could eat?"
]

gptconvo5 = [ #Games, Movies, Travel
    "User: Hey! Any favorite video games you like to play?",
    "Chatbot: I don't play games, but many enjoy action games. What's your favorite video game genre?",
    "User: Action games are intense! Speaking of intensity, have you seen any thrilling movies lately?",
    "Chatbot: I don't watch movies, but 'The Dark Knight' is often praised. What movie genres do you enjoy?",
    "User: 'The Dark Knight' is a masterpiece! I'm into sci-fi. On a different note, any favorite travel destinations?",
    "Chatbot: I don't travel, but people like Paris. What's your dream destination for a vacation?",
    "User: Paris is romantic! My dream destination is Tokyo. Have you ever considered virtual travel through VR games?",
    "Chatbot: Virtual travel sounds interesting! I don't play games, but VR technology is advancing. What are your thoughts on it?",
    "User: VR is a game-changer in gaming and beyond. The immersive experiences it offers are unparalleled. Speaking of which, have you tried any virtual reality experiences?",
    "Chatbot: I haven't experienced VR, but it's intriguing. The possibilities for education and entertainment are vast. What VR applications do you find most exciting?",
    "User: VR educational simulations are impressive. They provide hands-on learning experiences in various fields. Any particular VR application you'd be curious to explore?",
    "Chatbot: Medical VR simulations seem fascinating. They offer realistic training scenarios for healthcare professionals. What aspects of VR education do you find most promising?",
    "User: Medical VR is indeed promising, especially for surgical training. The ability to practice in a risk-free virtual environment is invaluable. Any other technological advancements you find exciting?",
    "Chatbot: The development of quantum computing holds immense potential. It could revolutionize fields like cryptography and optimization problems. What are your thoughts on the practical applications of quantum computing?",
    "User: Quantum computing is a frontier with limitless possibilities. Its impact on solving complex problems will reshape various industries. Anything else in the tech or gaming world you find intriguing?"
]

gptconvo6 = [ # Videos, Finance, Startups
    "User: Hi! Do you often watch online videos? Any favorite content creators?",
    "Chatbot: I don't watch videos, but many enjoy vlogs. Who's your favorite content creator?",
    "User: Vlogs are entertaining! Shifting gears, do you have any investment tips or financial advice?",
    "Chatbot: I don't provide financial advice, but diversifying investments is often recommended. What's your approach to financial planning?",
    "User: Diversification is key! On a different note, any interesting startups catching your attention lately?",
    "Chatbot: I don't follow startups, but tech startups are popular. Have you come across any promising ones?",
    "User: Tech startups are exciting! I'm keeping an eye on one in the AI space. By the way, any favorite finance books you'd recommend?",
    "Chatbot: I don't read, but 'Rich Dad Poor Dad' is often mentioned. What financial principles do you find most valuable?",
    "User: 'Rich Dad Poor Dad' is a classic! The emphasis on financial education is crucial. Speaking of which, do you participate in any online finance communities or forums?",
    "Chatbot: I don't participate, but communities like Reddit's Personal Finance can be informative. Do you find online forums helpful in sharing financial insights?",
    "User: Absolutely! Forums provide diverse perspectives and real-world experiences. It's a great way to learn from others' successes and mistakes. Have you ever explored personal finance apps for budgeting or investing?",
    "Chatbot: I don't use apps, but tools like Mint or Robinhood are popular for budgeting and investing. What's your go-to financial app?",
    "User: Mint is handy for budgeting! For investments, I use a combination of Robinhood and a more traditional brokerage. Have you ever considered investing in stocks or cryptocurrencies?",
    "Chatbot: I don't invest, but stocks and cryptocurrencies have gained attention. What's your strategy when it comes to investing in these markets?",
    "User: I follow a diversified approach, holding a mix of stocks and a small portion in cryptocurrencies for potential growth. It's essential to stay informed and adapt to market trends. Any specific sectors or industries you find promising for investment?",
    "Chatbot: Investing in tech companies seems logical, given their potential for innovation and growth. What tech sectors do you think hold the most promise in the coming years?",
    "User: Artificial intelligence and renewable energy are high on my list. Both have the potential to reshape industries. Speaking of innovation, have you heard about any groundbreaking technologies or startups lately?",
    "Chatbot: I'm not updated on specific startups, but biotech companies are often in the spotlight for their contributions to healthcare. What innovations in biotech or healthcare technology do you find most exciting?",
    "User: CRISPR gene editing technology is revolutionary in biotech. Its applications in treating genetic diseases show incredible potential. How about you? Any particular technology or startup you find intriguing?",
    "Chatbot: While I don't have personal preferences, advancements in clean energy technologies captivate many. Sustainable solutions could reshape our approach to energy consumption. What are your thoughts on the future of clean energy?",
    "User: Clean energy is crucial for a sustainable future. The integration of solar, wind, and other renewables is promising. It's inspiring to see the strides toward a greener planet. Anything else in finance, tech, or startups that piques your interest?"
]

gptconvo7 = [ # Anime, Travel, Health
    "User: Hello! Do you enjoy watching anime? Any favorite genres or series?",
    "Chatbot: I don't watch anime, but many enjoy action anime. What's your favorite anime genre?",
    "User: Action anime is thrilling! Shifting gears, do you like to travel? Any dream destinations?",
    "Chatbot: I don't travel, but people love tropical destinations. Where would you go for your dream vacation?",
    "User: Tropical destinations are paradise! My dream vacation is Bora Bora. Speaking of dreams, do you have any recurring dreams or nightmares?",
    "Chatbot: I don't dream, but people often share interesting dreams. What's the most vivid dream you've had recently?",
    "User: I had a dream about flying over a magical landscape. It felt so real! On a different note, do you follow any health and wellness practices?",
    "Chatbot: I don't have health practices, but many value mindfulness. What wellness activities do you incorporate into your routine?",
    "User: Mindfulness is important! I practice yoga for both physical and mental well-being. Have you ever tried yoga or meditation?",
    "Chatbot: I don't practice, but yoga and meditation have numerous benefits. What aspects of yoga do you find most beneficial?",
    "User: The combination of flexibility and relaxation is great! It helps with stress relief and maintaining a positive mindset. Have you explored any alternative health practices or holistic therapies?",
    "Chatbot: I don't explore therapies, but acupuncture and herbal remedies are popular. What alternative health practices interest you?",
    "User: Acupuncture is intriguing! I'm also curious about Ayurveda and traditional Chinese medicine. They focus on balancing the body's energies. Any health practices or therapies you find culturally interesting?",
    "Chatbot: Cultural practices influence health choices. Traditional Chinese medicine's holistic approach is fascinating. How do cultural perspectives on health impact your lifestyle choices?",
    "User: Cultural perspectives shape various aspects, including dietary choices and lifestyle habits. It's interesting to learn from diverse approaches. Anything else on your mind, or any other topics you'd like to explore?",
    "Chatbot: I don't have specific topics, but I'm here to chat and learn. Is there anything else you'd like to discuss or share?"
]

gptconvo8 = [ # History, Science, Hobbies
    "User: Hey! Are you interested in history? Any specific time periods or historical events you find intriguing?",
    "Chatbot: I don't have interests, but many find ancient civilizations fascinating. What historical era captures your attention?",
    "User: Ancient civilizations are rich in history! Shifting gears, are you into science? Any favorite scientific theories or discoveries?",
    "Chatbot: I don't have preferences, but the theory of relativity is often discussed. What scientific concepts do you find most captivating?",
    "User: The theory of relativity is mind-bending! On a different note, do you have any hobbies or recreational activities you enjoy?",
    "Chatbot: I don't have hobbies, but many enjoy reading or gaming. What hobbies bring you joy and relaxation?",
    "User: Reading and gaming are great choices! I also enjoy photography as a hobby. It allows me to capture moments and express creativity. Have you ever explored creative hobbies?",
    "Chatbot: I don't engage in hobbies, but creativity is valued. Photography sounds expressive. What aspects of photography do you find most rewarding?",
    "User: Capturing emotions and unique perspectives is what I love about photography. It's a visual storytelling medium. Shifting gears, any historical events or figures you find intriguing?",
    "Chatbot: I don't have preferences, but historical figures like Leonardo da Vinci are often admired. What qualities or achievements of historical figures do you find inspiring?",
    "User: Leonardo da Vinci was a polymath! His curiosity and contributions to art and science are truly inspiring. Speaking of science, any recent scientific discoveries that caught your attention?",
    "Chatbot: I don't keep up with discoveries, but advancements in space exploration are noteworthy. What space-related discoveries do you find most fascinating?",
    "User: Space exploration is incredible! The discovery of exoplanets and potential habitable zones is groundbreaking. The possibility of extraterrestrial life is intriguing. What are your thoughts on the search for extraterrestrial intelligence?",
    "Chatbot: The search for extraterrestrial intelligence is a captivating scientific pursuit. It raises questions about life beyond Earth. What aspects of space exploration or astronomy do you find most compelling?",
    "User: Exploring the mysteries of black holes and dark matter is fascinating. The vastness of the universe holds so many unknowns. Anything else you'd like to delve into, or any other topics that interest you?",
    "Chatbot: I'm open to any topics you'd like to discuss or explore. Is there anything specific on your mind, or any particular subject you'd like to know more about?"
]


gptconvo9 = [ # Fashion, Nature, Relationships
    "User: Hi! Are you interested in fashion? Any favorite fashion trends or designers?",
    "Chatbot: I don't have interests, but many follow trends like sustainable fashion. What fashion trends or styles resonate with you?",
    "User: Sustainable fashion is important! Shifting gears, do you enjoy spending time in nature? Any favorite outdoor activities or natural landscapes?",
    "Chatbot: I don't spend time outdoors, but people enjoy hiking in scenic landscapes. What outdoor activities or nature spots do you find most rejuvenating?",
    "User: Hiking is a great way to connect with nature! On a different note, how about relationships? Any advice or insights you find valuable?",
    "Chatbot: I don't give advice, but communication is often emphasized in relationships. What aspects of communication do you think are crucial for healthy relationships?",
    "User: Communication is key! Active listening and expressing feelings openly contribute to healthy relationships. Have you ever explored topics related to emotional intelligence or relationship psychology?",
    "Chatbot: I don't explore psychology, but emotional intelligence is recognized as important. How do you incorporate emotional intelligence into your relationships or interactions?",
    "User: Understanding emotions and being empathetic are central to emotional intelligence. It helps in building strong connections with others. Shifting gears, any fashion trends or styles you find intriguing?",
    "Chatbot: I don't have preferences, but minimalist fashion styles are often appreciated. What fashion styles or trends do you incorporate into your wardrobe?",
    "User: Minimalist styles are timeless! I also appreciate vintage fashion for its uniqueness. Speaking of nature, any natural wonders or landscapes you'd love to visit?",
    "Chatbot: I don't travel, but people often dream of visiting places like the Grand Canyon. What natural wonders or landscapes captivate your imagination?",
    "User: The Grand Canyon is breathtaking! I'd also love to visit the Northern Lights in Scandinavia. The beauty of nature is truly awe-inspiring. Any relationship advice or wisdom you've come across and found valuable?",
    "Chatbot: I don't collect wisdom, but understanding each other's love languages is often emphasized in relationships. What love languages resonate with you or have you found effective?",
    "User: Knowing and expressing love languages is crucial! Acts of service and quality time are important to me. Anything else you'd like to explore, or any other topics you're curious about?",
    "Chatbot: I'm here to chat and learn. Feel free to bring up any topics or questions you have in mind, and we can discuss further!"
]


gptconvo10 = [ # Education, Technology, Future
    "User: Hello! Are you interested in education? Any thoughts on modern education systems or teaching methods?",
    "Chatbot: I don't have opinions, but interactive learning approaches are gaining popularity. What educational methods do you think are effective?",
    "User: Interactive learning is engaging! Shifting gears, how about technology? Any futuristic technologies or inventions you're excited about?",
    "Chatbot: I don't anticipate, but advancements in artificial intelligence are often discussed. What AI applications or innovations do you find most promising?",
    "User: AI is transformative! The potential in healthcare, such as personalized medicine through AI, is groundbreaking. On a different note, do you have any favorite educational podcasts or channels?",
    "Chatbot: I don't have preferences, but educational podcasts like TED Talks cover diverse topics. What educational content or subjects do you enjoy exploring in your free time?",
    "User: TED Talks are insightful! I enjoy exploring topics in astrophysics and astronomy. The mysteries of the universe are captivating. Speaking of the future, any futuristic technologies you'd like to see become a reality?",
    "Chatbot: I don't wish, but clean energy solutions and sustainable technologies are ideal for the future. What futuristic technologies or innovations do you think would positively impact society?",
    "User: Sustainable technologies are crucial! I'd love to see advancements in eco-friendly transportation, like electric planes. Shifting gears, any thoughts on the role of technology in education?",
    "Chatbot: Technology enhances education by providing access to information and interactive learning tools. How do you think technology can further improve the educational experience?",
    "User: Technology can personalize learning, making it more adaptive to individual needs. Virtual reality simulations for hands-on experiences could also revolutionize education. Any tech trends or innovations you find revolutionary?",
    "Chatbot: The integration of augmented reality in education is gaining traction. It offers immersive learning experiences. What subjects or areas do you think would benefit the most from augmented reality applications?",
    "User: STEM subjects, like physics or biology, could benefit greatly. Visualizing complex concepts in a 3D environment enhances understanding. On a different note, do you have any favorite futuristic movies or books?",
    "Chatbot: I don't have favorites, but movies like 'Blade Runner' explore futuristic scenarios. What futuristic movies or books do you find thought-provoking?",
    "User: 'Blade Runner' is a classic! I also enjoy 'Ready Player One' for its portrayal of a virtual reality-driven future. Anything else on your mind, or any other topics you'd like to dive into?",
    "Chatbot: I'm open to any topics you'd like to discuss or explore. Is there anything specific on your mind, or any particular subject you'd like to know more about?"
]



# segments = semantic_splitter(encoder=encoder,docs=gptconvo2,threshold=0.60,split_method='consecutive_similarity_drop')
# segments2 = semantic_splitter(encoder=encoder2,docs=gptconvo4,threshold=0.71,split_method='cumulative_similarity_drop') ## threshold worked relatively well for convos 1 to 3 but only 1 split for convo 4
segments2 = semantic_splitter(encoder=encoder2,docs=gptconvo9,threshold=0.73,split_method='cumulative_similarity_drop')

visualize_topic_segments(segments2)
